# By Viraj A A Sanghai

In [1]:
#Use pandas package
import pandas
price_data = pandas.read_csv("Downloads/data_input_sample/stock_prices.csv")
#View stock prices data
price_data.head(5);


In [2]:
#View full stock prices data
#price_data

In [3]:
#Group stock prices by sector
price_data['sector'].value_counts();

In [4]:
#Group stock prices by industry
price_data['industry'].value_counts();

In [5]:
#looking at the different prices taken by the stocks
price_data['adjusted_price'].value_counts();

In [6]:
#Check for missing values in the stock prices data
price_data['adjusted_price'].isnull().sum();

In [7]:
#Find the length of time series for each security and finding the number of securities which is 529.
price_data['security_id'].value_counts();

## Source Features data preprocessing

In [8]:
#Load the source features data
source_features = pandas.read_csv("Downloads/data_input_sample/source_features.csv")
#print(source_features.head(5));

In [9]:
#Find the length of time series for source features
source_features['source_id'].value_counts();

In [10]:
#Index range of source features
source_features.index;

In [11]:
#After looking at the different data sets, we choose to focus on the source features. 
#We try to process it in a form that makes it more useable
import numpy as np
#As suggested in the guidelines, we reduce the 23,043 time series by grouping the data using category 4 and the unique report date
source_cat4 = source_features.groupby(['category_4','ReportDate']).agg({'source_id': np.sum,'source_features':sum})

In [12]:
#View sample of data - There are about 1387 category ids
source_cat4.head(5);

In [13]:
#source_cat4.index

In [14]:
#We are left with about 8003 time series with unique source ids and report dates
source_cat4['source_id'].value_counts();

In [15]:
#Since almost all securities prices data have time series with more than 500 points, 
#we keep only those time series source ids in the sourcecat4 data
boo_source = ((source_cat4['source_id'].value_counts())>=500) 
boo_source;

In [16]:
#We see that this will reduce our 8003 unique source ids to just 988
((source_cat4['source_id'].value_counts())>=500) .value_counts();

In [17]:
#We can double check this
boo_check = boo_source[boo_source.values == True]
boo_check;

In [18]:
#We can isin to filter out the source ids that we require from the source_cat4 data 
source_cat4 = source_cat4[(source_cat4['source_id']).isin(boo_check.index.tolist())]

In [19]:
#View the source_cat4 data
source_cat4.head(5);

In [20]:
#We can check the length of the time series of the left over data points
source_cat4['source_id'].value_counts();

In [21]:
#Again we can create groups by source ids and use  that as the main identifier for our time series
source_fin = source_cat4.groupby(['source_id'])
source_fin;

In [22]:
#We use the adfuller test to check for unit roots and stationarity of the 988 time series we have left. 
#We find we have 56 non-stationary time series. We plot them and count the the stationary and non statiopnary values in dictionaries.
#Although it is an extreme measure we use differencing to transform some of those time series to stationary data.
#After differencing once we are left with 9 non-stationary time series. After differencing twice we are left with 2 non-stationary time series.
#After differencing a third time we are left with one non-statioanry time series. 
#We drop the last remaining time series
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
%matplotlib inline
import matplotlib.pyplot as plt
source_stat = dict()
source_stat['stationary'] = 0
source_stat['not stationary'] = 0
source_diffstat = dict()
source_diffstat['diff stationary'] = 0
source_diffstat['diff not stationary'] = 0
source_diff2stat = dict()
source_diff2stat['diff2 stationary'] = 0
source_diff2stat['diff2 not stationary'] = 0
source_diff3stat = dict()
source_diff3stat['diff3 stationary'] = 0
source_diff3stat['diff3 not stationary'] = 0
not_stat  = []
for name, group in source_fin:
    #print(name)
    #print(group['source_features'])
    results= adfuller(group['source_features'])
    if results[1]<=0.05:
        #print(results[1], 'stationary')
        source_stat['stationary']+=1 
    else:
        #print(results[1], 'not stationary')
        source_stat['not stationary']+=1
        #(group['source_features']).plot()
        #plt.show()
        not_stat.append(name)
        #source_fin.drop(index = name)
        
       
            

    #(group['source_features']).plot()
    #print(results)
    #plt.show()

In [23]:
#Stationarity statistic of time series before differencing
source_stat;

In [24]:
#Stationarity statistic of time series after differencing once on non stationary data
not_stat;

In [25]:
#Stationarity statistic of time series after differencing twice on non stationary data
source_diff2stat;

In [26]:
#Stationarity statistic of time series after differencing thrice on non stationary data
source_diff3stat;

In [27]:
#A few checks in the next few inputs
#source_features['category_4'].value_counts()
#source_cat4.index

In [28]:
source_cat4['source_features'][0]['2017-02-01 00:00:00 ':'2017-02-05 00:00:00 '];

In [29]:
source_fin.head(5);

In [30]:
source_fin['source_id'].value_counts();

In [31]:
type(source_fin);

In [32]:
#source_fin.groups

In [33]:
#source_fin.get_group(107140)
#Source id no 22416 is the one still with non-stationary data. 
source_fin.get_group(22416);

In [34]:
source_cat4.index.get_level_values(1);

In [35]:
#We create a new dataframe using all our groups to keep only the essential data
#The report ids are the row indices, the source ids are the column labels 
#and the values are the values for the corresponding source features
final_source_df = pandas.DataFrame()

for name, group in source_fin:
    #print(name)
    #print(group['source_features'])
    #print(group.index.get_level_values(0))
    new_df = pandas.DataFrame({name: group['source_features'] })
    new_df.index = new_df.index.droplevel(0)
    #print(new_df)  
    #final_source_df = pandas.concat([final_source_df,new_df],axis=0)
    final_source_df = final_source_df.join(new_df,how = 'outer')
    #final_source_df = final_source_df.drop_duplicates()
 
#pandas.set_option('display.max_columns',1000)
#print(final_source_df)    

In [36]:
final_source_df[18].value_counts();

In [37]:
#Because we did an outer join, there are a few repeated rows which we can recombine
new_fin_df = final_source_df.groupby(level=0).mean()

In [38]:
#print(new_fin_df);

In [39]:
#We drop the non-stationary data column
new_fin_df.drop(not_stat, axis = 1, inplace = True)
#print(new_fin_df);

In [40]:
#We only keep rows that have 500 values
source_new = new_fin_df.dropna(thresh = 500)
#print(source_new);

In [41]:
#We throw away columns that have all NaN values
source_new = source_new.dropna(axis=1,how ='all')
#print(source_new);

In [42]:
#We interpolate the data to fill the NaN values with the mean of the columns
source_new2 = source_new.fillna(source_new.mean())
#print(source_new2)

In [43]:
price_data.head();

In [44]:
metal_df = price_data[price_data['name'] == price_data['name'][0]]
#print(metal_df)

In [45]:
metals_returns = metal_df[['adjusted_price','effective_date']]
metals_returns['adjusted_price'];

In [46]:
metals_returns.set_index('effective_date',inplace=True)
#metals_returns

In [47]:
metals_returns.loc[:,'adjusted_price'] = metals_returns['adjusted_price'].pct_change().values
#metals_returns

/Users/virajsanghai/python3/lib/python3.6/site-packages/pandas/core/indexing.py:621: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [48]:
metals_returns = metals_returns.dropna(axis=0)
metals_returns.rename(columns = {'adjusted_price':'returns'}, inplace=True) 
#metals_returns

In [49]:
source_new2.index;

In [50]:
metals_returns.index;

In [51]:
import pandas as pd
select_dates = pd.date_range(start = source_new2.index[0], end=source_new2.index[-1], freq ='B')
#np.array(select_dates)
sel = select_dates.strftime('%Y-%m-%d')
#sel


In [52]:
metals_returns_fin = metals_returns.loc[sel] #Use reindex instead
#metals_returns_fin

/Users/virajsanghai/python3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [53]:
metals_returns_fin.fillna(value = 0, inplace=True)

# First method: stationarity, feature scaling, lin regression, elastic net

In [54]:
#First split full data, then feature scale by fitting training data, then transforming test data
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(source_new2, metals_returns_fin, test_size = 0.2, random_state=0)
X_train, X_test = np.split(source_new2, [int(0.8*len(source_new2))])
y_train, y_test = np.split(metals_returns_fin, [int(0.8*len(metals_returns_fin))])
#y_train

In [55]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train.loc[:] = scaler.fit_transform(X_train)
X_test.loc[:] = scaler.transform(X_test)
#X_test

In [56]:
np.sum(metals_returns_fin.isnull());

In [57]:
#check met returns are stationary
met_result= adfuller(metals_returns_fin['returns'])
met_result[1];

In [58]:
#metals_returns_fin['returns'].plot()
#plt.show()

In [59]:
#Lin regression in statsmodel, can also use sklearn, make sure to add constant for statsmodels, 
#sklearn has a separate intercept and doesn't need a constant, sklearn better for metrics 
import statsmodels.api as sm
#print(hist_data.head())
#results2 = sm.OLS(y_train,X_train).fit_regularized(method='elastic_net',alpha=0.1,L1_wt = 0.5,refit = True)
#print(["{0:.15f}".format(i) for i in results2.params])
#print(results2.summary())

In [60]:
#Choose best alpha and l1 ratio for elastic net based on r^2
from sklearn.linear_model import ElasticNet
from sklearn.multioutput import MultiOutputRegressor

def build_model(_alpha, _l1_ratio):
    estimator = ElasticNet(
        alpha=_alpha,
        l1_ratio=_l1_ratio,
        fit_intercept=True,
        normalize=False,
        precompute=False,
        max_iter=1000,
        copy_X=True,
        tol=0.1,
        warm_start=False,
        positive=False,
        random_state=None,
        selection='random'
    )

    return MultiOutputRegressor(estimator, n_jobs=1)
    #return estimator

In [61]:
#Use time series split to maintain temporal order
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
#X_train, X_test, y_train, y_test = train_test_split(source_new2, metals_returns_fin, test_size = 0.2, random_state=0)

In [62]:
#Use gridsearch CV to find best parameters
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

params = {
    'estimator__alpha':(0.1, 0.3, 0.5, 0.7, 0.9),
    'estimator__l1_ratio':(0.1, 0.3, 0.5, 0.7, 0.9)
}

#for i in range(100):
model = build_model(_alpha=1.0, _l1_ratio=0.3)

finder = GridSearchCV(
    estimator=model,
    param_grid=params,
    scoring='r2',
    #fit_params=None,
    n_jobs=None,
    iid=False,
    refit=False,
    cv=tscv,  # change this to the splitter subject to test
    verbose=1,
    pre_dispatch=8,
    #error_score=-999,
    return_train_score=True
)

finder.fit(X_train, y_train)

best_params = finder.best_params_

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:    0.6s finished


In [63]:
best_params;

In [64]:
final_res = finder.cv_results_
final_res;

In [65]:
#Apply best fit model to data. Test using different metrics including r^2
from sklearn.metrics import r2_score
# optimal model
model = ElasticNet(
        alpha=0.1,
        l1_ratio=0.5,
        fit_intercept=True,
        normalize=False,
        precompute=False,
        max_iter=1000,
        copy_X=True,
        tol=1e-4,
        warm_start=False,
        positive=False,
        random_state=None,
        selection='cyclic'
    )
# train model
model.fit(X_train, y_train)
# test score
y_predicted = model.predict(X_test)
score = r2_score(y_test, y_predicted, multioutput='uniform_average')
#print("Test Loss: {0:.15f}".format(score))

In [66]:
#print(["{0:.15f}".format(i) for i in model.coef_])

# Method 2: stationarity, feature scaling, PCA, lin regression, elastic net

In [67]:
#For PCA first centre all data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
source_new2.iloc[:,0:128] = scaler.fit_transform(source_new2.iloc[:,0:128])

In [68]:
# Make sure sample size is greater than number of features
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(source_new2.iloc[:,0:128])
#print((pca.explained_variance_ratio_))

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [69]:
pca.components_.shape;

In [70]:
np.cumsum(pca.explained_variance_ratio_);

In [71]:
#Calculate principal components explain 90% variance
sum(np.cumsum(pca.explained_variance_ratio_)<=0.9);

In [72]:
#Another way to keep 90% variance of components
from sklearn.decomposition import PCA
pca_req = PCA(n_components=0.9,svd_solver='full')
pca_req.fit(source_new2.iloc[:,0:127])
#print((pca_req.components_.T))

PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
    svd_solver='full', tol=0.0, whiten=False)

In [73]:
#Recalculate transformed design matrix using principal eigenvectors
W = np.dot(source_new2.values[:,0:127], pca_req.components_.T)
W;

In [74]:
#Then split the data
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(source_new2, metals_returns_fin, test_size = 0.2, random_state=0)
W_train, W_test = np.split(W, [int(0.8*len(source_new2))])
y_train, y_test = np.split(metals_returns_fin, [int(0.8*len(metals_returns_fin))])
#y_train


In [75]:
#Use linear regression first
import statsmodels.api as sm
W_train_stats = sm.add_constant(W_train)
#print(hist_data.head())
results3 = sm.OLS(y_train,W_train_stats).fit()
#print(results3.summary())

In [76]:
#sklearn linear regression better for metrics and future predictions
from sklearn import linear_model
from sklearn.metrics import r2_score, accuracy_score, f1_score
regr = linear_model.LinearRegression()
results4 = regr.fit(W_train,y_train)
#print(regr.coef_)
y_predict1 =regr.predict(W_test)
score_lin_r = r2_score(y_test, y_predict1, multioutput='uniform_average')
#print("Test Loss: {0:.15f}".format(score_lin_r))

/Users/virajsanghai/python3/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [77]:
#print(regr.intercept_)

In [78]:
#Calculate accuracy and f1 score by times returns are positive or negative
bin_test = y_test > 0
bin_predict = y_predict1 > 0
score_lin_f1 = f1_score(bin_test, bin_predict)
#print("F1: {0:.15f}".format(score_lin_f1))

In [79]:
score_lin_acc = accuracy_score(bin_test, bin_predict)
#print("acc: {0:.15f}".format(score_lin_acc))

In [80]:
np.sum(bin_test == bin_predict);

In [81]:
#Calculate daily sharpe ratio against some benchmark and annualize it
sharpe_daily = np.mean(y_predict1 - 0)/np.std(y_predict1-0);

In [82]:
np.sqrt(260)*sharpe_daily;

In [83]:
#Use Elastic Net again, alpha = 1, l1 = 0, only ridge regression, l1 = 1 is lasso, l1 <0.01 not reliable
from sklearn.linear_model import ElasticNet
from sklearn.multioutput import MultiOutputRegressor

def build_model(_alpha, _l1_ratio):
    estimator = ElasticNet(
        alpha=_alpha,
        l1_ratio=_l1_ratio,
        fit_intercept=True,
        normalize=False,
        precompute=False,
        max_iter=1000,
        copy_X=True,
        tol=0.1,
        warm_start=False,
        positive=False,
        random_state=None,
        selection='random'
    )

    return MultiOutputRegressor(estimator, n_jobs=1)
    #return estimator

In [84]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
#X_train, X_test, y_train, y_test = train_test_split(source_new2, metals_returns_fin, test_size = 0.2, random_state=0)

In [85]:
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

params = {
    'estimator__alpha':(0.1, 0.3, 0.5, 0.7, 0.9,1),
    'estimator__l1_ratio':(0.1, 0.3, 0.5, 0.7, 0.9,1)
}

#for i in range(100):
model = build_model(_alpha=1.0, _l1_ratio=0.3)

finder2 = GridSearchCV(
    estimator=model,
    param_grid=params,
    scoring='r2',
    #fit_params=None,
    n_jobs=None,
    iid=False,
    refit=False,
    cv=tscv,  # change this to the splitter subject to test
    verbose=1,
    pre_dispatch=8,
    #error_score=-999,
    return_train_score=True
)

finder2.fit(W_train, y_train)

best_params2 = finder2.best_params_
best_params2;

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed:    0.6s finished


In [86]:
from sklearn.metrics import r2_score
# optimal model
model2 = ElasticNet(
        alpha=0.1,
        l1_ratio=0.05,
        fit_intercept=True,
        normalize=False,
        precompute=False,
        max_iter=1000,
        copy_X=True,
        tol=1e-4,
        warm_start=False,
        positive=False,
        random_state=None,
        selection='cyclic'
    )
# train model
model2.fit(W_train, y_train)
# test score
y_predict_pca = model2.predict(W_test)
score2 = r2_score(y_test, y_predict_pca, multioutput='uniform_average')
#print("Test Loss: {0:.15f}".format(score2))

In [87]:
#print(["{0:.15f}".format(i) for i in model2.coef_])

In [88]:
bin_test = y_test > 0
bin_predict_pca = y_predict_pca > 0
score_el_f1 = f1_score(bin_test, bin_predict_pca)
#print("F1: {0:.15f}".format(score_el_f1))

In [89]:
score_el_acc = accuracy_score(bin_test, bin_predict_pca)
#print("acc: {0:.15f}".format(score_el_acc))

In [90]:
sharpe_daily2 = np.mean(y_predict_pca - 0)/np.std(y_predict_pca-0);

In [91]:
np.sqrt(260)*sharpe_daily2;

In [92]:
import statsmodels.api as sm
#print(hist_data.head())
results5 = sm.OLS(y_train,W_train_stats).fit_regularized(method='elastic_net',alpha=0.1,L1_wt = 0,refit = True)
#print(["{0:.15f}".format(i) for i in results5.params])
#print(results5.summary())